In [ ]:
! pip install rdkit

In [ ]:
import torch
from torch_geometric.datasets import MoleculeNet

import numpy as np
import pandas as pd

import py3Dmol
from rdkit import Chem

import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pio.templates.default = "plotly_white"

dataset = MoleculeNet(root='data/MoleculeNet', name='Tox21')

print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')
print(f'Number of edge features: {dataset.num_edge_features}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

By looking at the target class, we can say this is multi-class problem!

In [ ]:
data.y

In [ ]:
def draw_molecule(mol):
    m = Chem.MolToMolBlock(mol, confId=-1)

    p = py3Dmol.view(width=400, height=400)
    p.removeAllModels()

    p.addModel(m, 'sdf')
    p.setStyle({'stack': {}})
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()

    return p.show()

In [ ]:
# Get a random graph from the dataset for inspection
i = 3666 # np.random.randint(len(dataset))
m = Chem.MolFromSmiles(dataset[i].smiles)
m

## Node features

* `Atomic number`: Number of protons in the nucleus of an atom. It’s characteristic of a chemical element and determines its place in the periodic table.

* `Chirality`: A molecule is chiral if it is distinguishable from its mirror image by any combination of rotations, translations, and some conformational changes. Different types of chirality exist depending on the molecule and the arrangement of the atoms.

* `Degree`: Number of directly-bonded neighbors of the atom.

* `Formal charge`: Charge assigned to an atom. It reflects the electron count associated with the atom compared to the isolated neutral atom.

* `Number of H`: Total number of hydrogen atoms on the atom.

* `Number of radical e`: Number of unpaired electrons of the atom.

* `Hybridization`: Atom’s hybridization.

* `Is aromtic`: Whether it is included in a cyclic structure with pi bonds. This type of structure tends to be very stable in comparison with other geometric arrangements of the same atoms.

* `Is in ring`: Whether it is included in a ring (a simple cycle of atoms and bonds in a molecule).


In [ ]:
for i in range(len(dataset)):
    x_i = dataset[i].x.cpu().detach().numpy()
    x = x_i if i ==0 else np.vstack([x,dataset[i].x.cpu().detach().numpy()])

    y_i = dataset[i].y.cpu().detach().numpy()
    y = y_i if i ==0 else np.vstack([y,dataset[i].y.cpu().detach().numpy()])

df_x = pd.DataFrame(x)

In [ ]:
print(f'All the node features for the dataset: {df_x.shape}')
df_x.head()

In [ ]:
df_x.columns = [
    'atomic_num', 'chirality', 'degree', 'formal_charge',
    'numH', 'number_radical_e', 'hybridization',
    'is_aromatic', 'is_in_ring'
]

for col in df_x:
    px.histogram(
        df_x, col, histnorm='percent',
        height=300, width=500, title='Distribution of '+col).show()

## Edge features

* `Bond type`:  Whether the bond is single, double, triple, or aromatic.

* `Stereo configuration`: stereo configuration of the bond.

* `Is conjugated`: Whether o not bond is considered to be conjugated.

**aromatic compound, any of a large class of unsaturated chemical compounds characterized by one or more planar rings of atoms joined by covalent bonds of two different kinds. The unique stability of these compounds is referred to as aromaticity.**

In [ ]:
for i in range(len(dataset)):
    x_i = dataset[i].edge_attr.cpu().detach().numpy()
    x = x_i if i ==0 else np.vstack([x,dataset[i].edge_attr.cpu().detach().numpy()])

df_edge = pd.DataFrame(x)
df_edge.columns = ['bond_type', 'sterio_configuration', 'is_conjugated']

In [ ]:
for col in df_edge:
    px.histogram(
        df_edge, col, histnorm='percent',
        height=300, width=500, title='Distribution of '+ col).show()

Only `bond_type` and `stero_configuration` features are imbalanced!

## What are targets?

The dataset contains the outcomes of 12 different toxicological experiments in the form of binary labels (active/inactive).

In [ ]:
df_y = pd.DataFrame(y).melt()

df_y1 = df_y.groupby(['variable'],as_index=False).agg({'value':['sum', 'count']})
df_y1.columns = ['experiment','sum', 'count', ]
df_y1['%_of_toxic_m'] = df_y1['sum']/df_y1['count']
df_y1['missing_values'] = (1 - df_y1['count'] / len(dataset))
df_y1['perc_of_samples'] = df_y1['count'] / len(dataset)

for c in ['%_of_toxic_m', 'missing_values', 'perc_of_samples', ]:
    df_y1[c] = df_y1[c].apply(lambda x:round(x*100,2))

print(df_y1.shape)
df_y1

In [ ]:
fig = px.line(df_y1, 'experiment', '%_of_toxic_m', 
              title='Number of Positive Toxic Examples In Experiments', 
              text='%_of_toxic_m')
fig.show()

The data poses two main challenges:
* **Small dataset**: The number of labeled molecules varies depending on the experiment
* **Unbalanced target**: The percentage of active molecules is very low, up to 3% as above table

# Feature engineering

In [ ]:
cols_to_normalize = [
    'atomic_num', 'degree',
    'formal_charge',
    'numH',
    'number_radical_e'
]

cols_to_encode = [
    'chirality',
    'hybridization'
]

METHOD = 'min-max'

scalers = {}

for c in cols_to_normalize + cols_to_encode:
    if METHOD == 'normal':
        scalers[c] = {'mean':df_x[c].mean(), 'std':df_x[c].std()}
    if METHOD == 'min-max':
        scalers[c] = {'min': df_x[c].min(), 'max': df_x[c].max()}

# 1 => single bond
# 2 => double bond
# 3 => triple bond
# 3 > 0 => aromatic (ring structure)
scalers['bond_type'] = {'min': 1, 'max': 12}

In [ ]:
scalers

In [ ]:
dataset_new = []

for i in range(len(dataset)):
    data = dataset[i]
    x_norm = data.x.detach().cpu().numpy().astype(float)
    edge_w_norm = data.edge_attr[:,0].detach().cpu().numpy().astype(float)
    edge_a_norm = data.edge_attr[:,0].detach().cpu().numpy().astype(int)

    # normalize columns
    for c in cols_to_normalize:
        col_i = list(df_x.columns).index(c)

        if METHOD == 'normal':
            x_norm[:, col_i] = (x_norm[:, col_i] - scalers[c]['mean'])/scalers[c]['std']
        if METHOD == 'min-max':
            x_norm[:, col_i] = (x_norm[:, col_i] - scalers[c]['min'])/(scalers[c]['max'] - scalers[c]['min'])

    # one-hot encoding of categorical columns
    for i,c in enumerate(cols_to_encode):

        col = x_norm[:,list(df_x.columns).index(c)].astype(int)
        col_enc = np.zeros((col.size, scalers[c]['max']+1))
        col_enc[np.arange(col.size), col] = 1

        cols_encoded = col_enc if i == 0 else np.hstack([cols_encoded, col_enc])

    cols_i_to_encode = [list(df_x.columns).index(c) for c in cols_to_encode]
    x_norm = x_norm[:,[i for i in range(x_norm.shape[1]) if i not in cols_i_to_encode]]
    x_norm = np.hstack([x_norm, cols_encoded])

    # normalize type of bonds
    edge_w_norm = (edge_w_norm - scalers['bond_type']['min'])/(scalers['bond_type']['max'] - scalers['bond_type']['min'])

    # one-hot encoding of type of bonds
    edge_a_norm = data.edge_attr[:,0].detach().cpu().numpy().astype(int)
    col_enc = np.zeros((edge_a_norm.size, scalers['bond_type']['max']+1))
    col_enc[np.arange(edge_a_norm.size), edge_a_norm] = 1

    # saving results
    data.x_norm = torch.tensor(x_norm, dtype=torch.float)
    data.edge_w_norm = torch.tensor(edge_w_norm, dtype=torch.float)
    data.edge_a_norm = torch.tensor(col_enc, dtype=torch.float)

    dataset_new.append(data)

In [ ]:
dataset_new[0]

In [ ]:
TARGET = 2 # Taking class 2

dataset_target, Y = [], []

for i in range(len(dataset_new)):
    if not(dataset_new[i]['y'][0,TARGET].isnan()):
        Y.append(dataset_new[i]['y'][0,TARGET])
        dataset_target.append(dataset_new[i])

Y = pd.DataFrame([y.numpy() for y in Y]).reset_index().rename(columns={0:'target'})

print(f'Average of the target: {Y.target.mean() * 100}')

In [ ]:
Y